In [1]:
import os

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("groq:qwen/qwen3-32b")
model.invoke("what is artificial intelligence")

AIMessage(content='<think>\nOkay, the user is asking, "What is artificial intelligence?" Let me start by breaking this down. AI is a broad term, so I need to define it clearly. Maybe start with a basic definition.\n\nFirst, AI refers to systems designed to perform tasks that typically require human intelligence. But what are those tasks? Examples like problem-solving, learning, perception, language understanding come to mind. I should list some of these to make it concrete.\n\nNext, I should mention the different areas or subfields within AI. Machine learning is a big one, where systems learn from data. Then there\'s natural language processing for understanding human language, computer vision for image recognition, robotics, and expert systems. Maybe explain each briefly.\n\nIt\'s important to highlight the goals of AI. Are there different approaches? Like symbolic AI versus machine learning? Or maybe explain the distinction between narrow AI and general AI. Narrow AI is what\'s in us

In [3]:
from pydantic import BaseModel,Field
class Movie(BaseModel):
    title:str=Field(description="The title of the movie is")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie is")
    rating:float=Field(description="The movie rating out of 10")

In [4]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002D18E152510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D18E153230>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie is', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie is', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'required': ['title', 'yea

In [8]:
model_with_structure.invoke("provide details about the movie Bahubali")

Movie(title='Bahubali', year=2015, director='S.S. Rajamouli', rating=8.3)

In [9]:
from pydantic import BaseModel,Field
class Movie(BaseModel):
    """A movie with details"""
    title: str = Field(...,description="The title of the movie")
    year: str = Field(...,description="This year the movie was released")
    director: str = Field(...,description="The director of the movie is")
    rating: str = Field(...,description="The movie rating out of 10")

model_with_structure = model.with_structured_output(Movie,include_raw=True)

In [12]:
response=model_with_structure.invoke("Provide details about the movie Bahubali")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Bahubali. Let me check the tools provided. There\'s a function called Movie that requires title, year, director, and rating. I need to fill those parameters. I know Bahubali is a popular Indian film. The title is "Bahubali: The Beginning" and the sequel is "Bahubali 2: The Conclusion". The director is S.S. Rajamouli. The release year for the first one was 2015. The rating on IMDb is around 8.0. Let me verify these details to make sure. Yes, that\'s correct. So I\'ll structure the tool call with these parameters.\n', 'tool_calls': [{'id': '8cv26bkny', 'function': {'arguments': '{"director":"S.S. Rajamouli","rating":"8.0","title":"Bahubali: The Beginning","year":"2015"}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 232, 'total_tokens': 433, 'completion_time': 0.360242791, 'completion_tokens_d

### Nested Structure

In [13]:
from pydantic import BaseModel,Field
class Actor(BaseModel):
    name:str
    role:str
class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")

In [18]:
model_with_structure = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide the details about the movie Bahubali")

In [19]:
response

MovieDetails(title='Bahubali', year=2015, cast=[Actor(name='Prabhas', role='Baahubali'), Actor(name='Rana Daggubati', role='Amarendra Baahubali'), Actor(name='Tamannaah Bhatia', role='Devasena')], genres=['Action', 'Adventure', 'Fantasy'], budget=10.0)

In [30]:
from typing_extensions import TypedDict,Annotated
class MovieDict(TypedDict):
    """A movie with details"""
    title: Annotated[str,...,"The title of the movie"]
    year: Annotated[int,..., "The year the movie was released"]
    director: Annotated[str,..., "The directir of the movie is"]
    rating: Annotated[float,..., "The movie rating out of 10"]
    hero: Annotated[str,..., "The hero of the movie"]
    heroine: Annotated[str,..., "The herione of the movie"]

model_withtyped_dict = model.with_structured_output(MovieDict)

In [37]:
response = model_withtyped_dict.invoke("Provide the details about the movie Bahubali")
response

{'director': 'S.S. Rajamouli',
 'hero': 'Prabhas',
 'heroine': 'Tamannaah',
 'rating': 8.5,
 'title': 'Bahubali',
 'year': 2015}

In [38]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}